In [1]:
import pandas as pd
import numpy as np

In [2]:
raw = pd.read_csv("./raw_stars.csv")
raw

,id,hip,hd,hr,gl,bf,proper,ra,dec,dist,...,bayer,flam,con,comp,comp_primary,base,lum,var,var_min,var_max
0,0,NaN,NaN,NaN,NaN,NaN,Sol,0.000000,0.000000,0.0000,...,NaN,NaN,NaN,1,0,NaN,1.000000,NaN,NaN,NaN
1,1,1.0,224700.0,NaN,NaN,NaN,NaN,0.000060,1.089009,219.7802,...,NaN,NaN,Psc,1,1,NaN,9.638290,NaN,NaN,NaN
2,2,2.0,224690.0,NaN,NaN,NaN,NaN,0.000283,-19.498840,47.9616,...,NaN,NaN,Cet,1,2,NaN,0.392283,NaN,NaN,NaN
3,3,3.0,224699.0,NaN,NaN,NaN,NaN,0.000335,38.859279,442.4779,...,NaN,NaN,And,1,3,NaN,386.901132,NaN,NaN,NaN
4,4,4.0,224707.0,NaN,NaN,NaN,NaN,0.000569,-51.893546,134.2282,...,NaN,NaN,Phe,1,4,NaN,9.366989,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119609,119611,NaN,NaN,NaN,NN 4381,NaN,NaN,23.963895,38.629391,16.9492,...,NaN,NaN,NaN,1,119611,NaN,0.002200,NaN,NaN,NaN
119610,119612,NaN,NaN,NaN,NN 4385,NaN,NaN,23.996567,47.762093,16.7224,...,NaN,NaN,NaN,1,119612,NaN,0.000088,NaN,NaN,NaN
119611,119613,NaN,NaN,NaN,NN 4386,NaN,NaN,23.996218,-44.067905,18.5185,...,NaN,NaN,NaN,1,119613,NaN,0.002224,NaN,NaN,NaN
119612,119614,NaN,NaN,NaN,NN 4387,NaN,NaN,23.997386,-34.111986,12.8205,...,NaN,NaN,NaN,1,119614,NaN,0.001086,NaN,NaN,NaN


In [3]:
named = raw.rename(columns={"proper": "name"})
named["name"].fillna(named["bf"].str.replace(" +", " ", regex=True), inplace=True)
#named["name"] = named["name"].str.replace(r"^[0-9]+", r"", regex=True)
named = named[["name", "x", "y", "z", "spect"]]
named.dropna(inplace=True)
named

,name,x,y,z,spect
0,Sol,0.000005,0.000000,0.000000,G2V
88,Tau Phe,119.736878,0.562356,-136.823418,G8III
122,The Oct,14.902031,0.103716,-64.888859,K2III
145,29 Psc,139.077212,1.106982,-7.355907,B7III-IV
154,30 Psc,131.716394,1.126662,-13.877167,M3III
...,...,...,...,...,...
119133,36 Oph,-1.036233,-5.250282,-2.680239,K1 Ve
119193,69Tau Oph,0.690292,-51.492963,-7.404135,dF3 J
119257,Gam CrA,4.078969,-13.679989,-10.782412,F8 V
119431,78Mu 2Cyg,16.290512,-10.974411,10.773027,G2 V


In [15]:
nearby = named.copy()
nearby["dist"] = (nearby.x**2 + nearby.y**2 + nearby.z**2)**0.5
nearby.sort_values(["dist"], inplace=True)
nearby[["x", "y", "z"]] = nearby[["x", "y", "z"]].round(2)

dist = 100

nearby = nearby[(nearby["x"].abs() < dist) & (nearby["y"].abs() < dist) & (nearby["z"].abs() < 10)]
nearby.drop(columns="dist", inplace=True)
nearby

,name,x,y,z,spect
0,Sol,0.00,0.00,0.00,G2V
70666,Proxima Centauri,-0.47,-0.36,-1.15,M5Ve
71453,Alp2Cen,-0.50,-0.41,-1.16,K1V
71456,Rigil Kentaurus,-0.50,-0.41,-1.16,G2V
87665,Barnard's Star,-0.02,-1.82,0.15,sdM4
...,...,...,...,...,...
17062,12 Tau,65.37,93.23,6.08,G6III:
42283,5Sig Hya,-72.86,87.80,6.66,K2III
74421,3 Ser,-75.09,-85.77,9.85,K0III
105426,21 Aqr,92.76,-74.26,-7.39,K4III


In [16]:
typed = nearby.copy()
typed["size"] = typed["spect"].replace(r".*([0VI]+).*", r"\1", regex=True).replace(r"[^0VI]", np.nan, regex=True)
typed["spect"] = typed["spect"].replace(r".*([OBAFGKMLT][0-9]).*", r"\1", regex=True)
typed.dropna(inplace=True)
typed["id"] = np.arange(len(typed))
typed = typed[["id", "name", "x", "y", "z", "spect", "size"]]
typed

,id,name,x,y,z,spect,size
0,0,Sol,0.00,0.00,0.00,G2,V
70666,1,Proxima Centauri,-0.47,-0.36,-1.15,M5,V
71453,2,Alp2Cen,-0.50,-0.41,-1.16,K1,V
71456,3,Rigil Kentaurus,-0.50,-0.41,-1.16,G2,V
53879,4,Lalande 21185,-2.00,0.50,1.50,M2,V
...,...,...,...,...,...,...,...
17062,444,12 Tau,65.37,93.23,6.08,G6,I
42283,445,5Sig Hya,-72.86,87.80,6.66,K2,I
74421,446,3 Ser,-75.09,-85.77,9.85,K0,I
105426,447,21 Aqr,92.76,-74.26,-7.39,K4,I


In [17]:
abs = typed.copy()
abs[["x", "y", "z"]] -= abs[["x", "y", "z"]].min()
abs

,id,name,x,y,z,spect,size
0,0,Sol,96.65,99.91,9.84,G2,V
70666,1,Proxima Centauri,96.18,99.55,8.69,M5,V
71453,2,Alp2Cen,96.15,99.50,8.68,K1,V
71456,3,Rigil Kentaurus,96.15,99.50,8.68,G2,V
53879,4,Lalande 21185,94.65,100.41,11.34,M2,V
...,...,...,...,...,...,...,...
17062,444,12 Tau,162.02,193.14,15.92,G6,I
42283,445,5Sig Hya,23.79,187.71,16.50,K2,I
74421,446,3 Ser,21.56,14.14,19.69,K0,I
105426,447,21 Aqr,189.41,25.65,2.45,K4,I


In [18]:
abs.to_csv("stars.csv", index=False)
abs.to_json("stars.json", "records")